# PreTraining Undamaged Model

Objective: train a YOLO model on pre-event undamaged buildings (residential, commercial) to a sufficient level of accuracy. This model will be trained on an initial manually annotated dataset.

In [1]:
import os
HOME = os.getcwd()
print(f'HOME directory path: \'{HOME}\'')

HOME directory path: '/content'


#### Initialization of Google Collab environment

In [2]:
requirements_packages = """
rasterio==1.4.2
ultralytics>=8.3
labelme2yolo==0.2.5
roboflow==1.1.48
"""

# Specify the file name
file_name = "requirements.txt"

# Write the text to the file
with open(file_name, 'w') as file:
    file.write(requirements_packages)

print(f"File '{file_name}' has been created with the text.")

File 'requirements.txt' has been created with the text.


In [3]:
# Installing required packages quietly without showing the details
!pip install -r {HOME}/requirements.txt --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.

# Connecting to RoboFlow database

In [4]:
import roboflow
roboflow.login(force=True) # Login into roboflow

visit https://app.roboflow.com/auth-cli to get your authentication token.
Paste the authentication token here: ··········


## Importing packages

In [5]:
import torch

# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# GeoTiff Images
import rasterio
from osgeo import gdal

# Visualisation
from matplotlib import pyplot as plt
import matplotlib.image as img
from matplotlib.pyplot import figure
from PIL import Image

# Model Building
import ultralytics
from ultralytics import YOLO
from IPython.display import Image

# Progress bar
from tqdm import tqdm

import numpy as np

# Others
import os
import shutil
import zipfile

%matplotlib inline

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


## Define project details

In [6]:
# Downloading dataset from Roboflow platform (Notice: after execution data will be deleted from google colab)
roboflow_project_name = 'stormdamagechallenge'
dataset_name = 'storm-damage'
dataset_title = 'Storm-Damage'
dataset_id = 'goe2a'
dataset_version = 14 # Cleaned Commercial Buildings
model_name = 'yolo'
model_version = '11' # 'v8' or '11'
model_variant = 'n' # parameter required for model training phase

# Connect to roboflow dataset
rf = roboflow.Roboflow()
project = rf.workspace(roboflow_project_name).project(dataset_name + '-' + dataset_id)
dataset = project.version(dataset_version).download(model_name + model_version)

loading Roboflow workspace...
loading Roboflow project...

Version export complete for yolo11 format



Extracting Dataset Version Zip to Storm-Damage-14 in yolo11:: 100%|██████████| 774/774 [00:00<00:00, 5526.32it/s]


### Check Ultralytics connection

In [7]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
ultralytics.checks()

Ultralytics 8.3.178 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 43.2/112.6 GB disk)


In [8]:
!nvidia-smi

Tue Aug 12 18:32:02 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### Loading YOLO model for Training

In [10]:
model = YOLO("/content/best_model/best.pt")  # load a custom trained model
model.info()

YOLO11n summary: 181 layers, 2,590,230 parameters, 0 gradients, 6.4 GFLOPs


(181, 2590230, 0, 6.4416768)

In [11]:
# # Loading the model
# model = YOLO(f'../models/{model_name + model_version}/{model_name + model_version + model_variant}.pt')
# # Display model information (optional)
# model.info()

In [ ]:
yaml_file = f'{HOME}/{dataset_title}-{dataset_version}/data.yaml'
yaml_file

'/content/Storm-Damage-14/data.yaml'

## Pre-Training Model

In [12]:
# Define model training parameters
batch = -1 # set as an integer (e.g., batch=16), auto mode for 60% GPU memory utilization (batch=-1), or auto mode with specified utilization fraction (batch=0.70)
device = 0 # 0: GPU | 'cpu': cpu | 'mps': apple silicon mps
epochs = 50
patience = 10
imgsz = 640 # TODO: assess the importance of image size 640x640 by-design of Yolo models, image size role
plots = True
project = 'exploratory_phase' # parent directory name
name_train = f'{model_name + model_version + model_variant}_training_dataset_v{dataset_version}' # run directory name

In [ ]:
# # Train the model
# result_model_training = model.train(
#                                     data=yaml_file,
#                                     batch = batch,
#                                     device = device,
#                                     epochs = epochs,
#                                     patience = patience,
#                                     imgsz = imgsz,
#                                     optimizer = 'AdamW',
#                                     # augment = False, # (bool) apply image augmentation to prediction sources
#                                     lr0 = 1e-5,
#                                     # auto_augment = None, # (str) auto augmentation policy for classification (randaugment, autoaugment, augmix)
#                                     momentum = 0.937,
#                                     plots = plots,
#                                     project = project,
#                                     name = name_train
#                                     )

Ultralytics 8.3.174 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Storm-Damage-14/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=1e-05, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=../models/yolo11/yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11n_training_dataset_v14, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=10, perspe

Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

YOLO11n summary: 181 layers, 2,590,230 parameters, 2,590,214 gradients, 6.4 GFLOPs

Transferred 448/499 items from pretrained weights
Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1477.7±669.1 MB/s, size: 56.6 KB)


train: Scanning /content/Storm-Damage-14/train/labels... 333 images, 1 backgrounds, 0 corrupt: 100%|██████████| 333/333 [00:00<00:00, 2120.17it/s]

...

      50/50      8.98G      1.677      1.505      1.458        265        640: 100%|██████████| 7/7 [00:04<00:00,  1.73it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.00it/s]

                   all         32        842      0.561      0.622      0.571      0.268
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 40, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



50 epochs completed in 0.093 hours.
Optimizer stripped from exploratory_phase/yolo11n_training_dataset_v14/weights/last.pt, 5.5MB
Optimizer stripped from exploratory_phase/yolo11n_training_dataset_v14/weights/best.pt, 5.5MB

Validating exploratory_phase/yolo11n_training_dataset_v14/weights/best.pt...
Ultralytics 8.3.174 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.55it/s]


                   all         32        842      0.567      0.632      0.597       0.29
undamagedcommercialbuilding         14         58      0.507      0.517      0.456      0.244
undamagedresidentialbuilding         29        784      0.626      0.746      0.738      0.335
Speed: 0.2ms preprocess, 3.2ms inference, 0.0ms loss, 5.0ms postprocess per image
Results saved to exploratory_phase/yolo11n_training_dataset_v14


In [ ]:
metrics = model.val()
metrics.box.map # map50-95
metrics.box.map50 # map50
metrics.box.map75 # map75
metrics.box.maps # a list contains map50-95 of each category

Ultralytics 8.3.174 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 946.6±531.1 MB/s, size: 55.8 KB)


val: Scanning /content/Storm-Damage-14/valid/labels.cache... 32 images, 0 backgrounds, 0 corrupt: 100%|██████████| 32/32 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.17s/it]


                   all         32        842      0.564      0.631      0.597      0.289
undamagedcommercialbuilding         14         58      0.503      0.517      0.457      0.241
undamagedresidentialbuilding         29        784      0.625      0.745      0.738      0.336
Speed: 0.3ms preprocess, 11.1ms inference, 0.0ms loss, 5.8ms postprocess per image
Results saved to exploratory_phase/yolo11n_training_dataset_v142


array([    0.24141,     0.33633])

In [ ]:
metrics = model.val()
metrics.box.map # map50-95
metrics.box.map50 # map50
metrics.box.map75 # map75
metrics.box.maps # a list contains map50-95 of each category

Ultralytics 8.3.174 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1180.4±390.2 MB/s, size: 55.8 KB)


val: Scanning /content/Storm-Damage-14/valid/labels.cache... 32 images, 0 backgrounds, 0 corrupt: 100%|██████████| 32/32 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.17s/it]


                   all         32        842      0.651      0.555      0.591      0.289
undamagedcommercialbuilding         14         58      0.576      0.445      0.441      0.241
undamagedresidentialbuilding         29        784      0.726      0.665      0.741      0.336
Speed: 0.3ms preprocess, 6.2ms inference, 0.0ms loss, 8.3ms postprocess per image
Results saved to exploratory_phase/yolo11n_training_dataset_v142


array([    0.24123,     0.33584])

In [ ]:
metrics.box.map50

np.float64(0.5971192128868948)

In [ ]:
from google.colab import files
files.download("/content/exploratory_phase/yolo11n_training_dataset_v14")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
metrics.box.fitness

<bound method Metric.fitness of ultralytics.utils.metrics.Metric object with attributes:

all_ap: array([[    0.45671,     0.43871,     0.40709,     0.37534,       0.254,     0.24833,     0.14325,    0.086312,    0.003748,  0.00055324],
       [    0.73753,     0.68615,     0.62947,     0.52417,     0.41966,     0.23434,    0.098294,    0.028749,   0.0044693,  0.00046291]])
ap: array([    0.24141,     0.33633])
ap50: array([    0.45671,     0.73753])
ap_class_index: array([0, 1])
curves: []
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048,    0.049049,     0.05005,    0.051051,    0.052052,    0.053053,    0.054054,    0.055055,    0.056056,    0.057057,    0.058058,    0.059059,     0.06006,    0.061061,    0.062062,    0.063063,    0.064064,    0.065065,    0.066066,    0.067067,    0.068068,    0.069069,     0.07007,    0.071071,
          0.072072,    0.073073,    0.074074,    0.075075,    0.076076,    0.077077,    0.078078,    0.079079,     0.08008,    0.081081,    0.082082,    0.083083,    0.084084,    0.085085,    0.086086,    0.087087,    0.088088,    0.089089,     0.09009,    0.091091,    0.092092,    0.093093,    0.094094,    0.095095,
          0.096096,    0.097097,    0.098098,    0.099099,      0.1001,      0.1011,      0.1021,      0.1031,      0.1041,     0.10511,     0.10611,     0.10711,     0.10811,     0.10911,     0.11011,     0.11111,     0.11211,     0.11311,     0.11411,     0.11512,     0.11612,     0.11712,     0.11812,     0.11912,
           0.12012,     0.12112,     0.12212,     0.12312,     0.12412,     0.12513,     0.12613,     0.12713,     0.12813,     0.12913,     0.13013,     0.13113,     0.13213,     0.13313,     0.13413,     0.13514,     0.13614,     0.13714,     0.13814,     0.13914,     0.14014,     0.14114,     0.14214,     0.14314,
           0.14414,     0.14515,     0.14615,     0.14715,     0.14815,     0.14915,     0.15015,     0.15115,     0.15215,     0.15315,     0.15415,     0.15516,     0.15616,     0.15716,     0.15816,     0.15916,     0.16016,     0.16116,     0.16216,     0.16316,     0.16416,     0.16517,     0.16617,     0.16717,
           0.16817,     0.16917,     0.17017,     0.17117,     0.17217,     0.17317,     0.17417,     0.17518,     0.17618,     0.17718,     0.17818,     0.17918,     0.18018,     0.18118,     0.18218,     0.18318,     0.18418,     0.18519,     0.18619,     0.18719,     0.18819,     0.18919,     0.19019,     0.19119,
           0.19219,     0.19319,     0.19419,      0.1952,      0.1962,      0.1972,      0.1982,      0.1992,      0.2002,      0.2012,      0.2022,      0.2032,      0.2042,     0.20521,     0.20621,     0.20721,     0.20821,     0.20921,     0.21021,     0.21121,     0.21221,     0.21321,     0.21421,     0.21522,
           0.21622,     0.21722,     0.21822,     0.21922,     0.22022,     0.22122,     0.22222,     0.22322,     0.22422,     0.22523,     0.22623,     0.22723,     0.22823,     0.22923,     0.23023,     0.23123,     0.23223,     0.23323,     0.23423,     0.23524,     0.23624,     0.23724,     0.23824,     0.23924,
           0.24024,     0.24124,     0.24224,     0.24324,     0.24424,     0.24525,     0.24625,     0.24725,     0.24825,     0.24925,     0.25025,     0.25125,     0.25225,     0.25325,     0.25425,     0.25526,     0.25626,     0.25726,     0.25826,     0.25926,     0.26026,     0.26126,    

## Use trained model for inference

In [13]:
os

<module 'os' (frozen)>

In [17]:
images_inference = ['/content/images_inference/'+filename for filename in os.listdir("/content/images_inference")]
images_inference

['/content/images_inference/Pre_Event_tile_1_68.jpg',
 '/content/images_inference/Pre_Event_tile_0_51.jpg',
 '/content/images_inference/Pre_Event_tile_0_76.jpg',
 '/content/images_inference/Pre_Event_tile_1_80.jpg',
 '/content/images_inference/Pre_Event_tile_1_58.jpg',
 '/content/images_inference/Pre_Event_tile_0_92.jpg',
 '/content/images_inference/Pre_Event_tile_1_34.jpg',
 '/content/images_inference/Pre_Event_tile_0_14.jpg',
 '/content/images_inference/Pre_Event_tile_1_92.jpg',
 '/content/images_inference/Pre_Event_tile_0_96.jpg',
 '/content/images_inference/Pre_Event_tile_0_25.jpg',
 '/content/images_inference/Pre_Event_tile_1_4.jpg',
 '/content/images_inference/Pre_Event_tile_0_0.jpg',
 '/content/images_inference/Pre_Event_tile_0_41.jpg',
 '/content/images_inference/Pre_Event_tile_1_75.jpg',
 '/content/images_inference/Pre_Event_tile_1_95.jpg',
 '/content/images_inference/Pre_Event_tile_0_86.jpg',
 '/content/images_inference/Pre_Event_tile_0_11.jpg',
 '/content/images_inference/Pr

In [18]:
# list_images = ["/content/Pre_Event_tile_2_24.jpg", "/content/Pre_Event_tile_2_26.jpg"]
inference_results = model.predict(images_inference, save=False, imgsz=640, conf=0.5)


0: 640x640 1 undamagedcommercialbuilding, 3.6ms
1: 640x640 55 undamagedresidentialbuildings, 3.6ms
2: 640x640 (no detections), 3.6ms
3: 640x640 (no detections), 3.6ms
4: 640x640 17 undamagedresidentialbuildings, 3.6ms
5: 640x640 3 undamagedresidentialbuildings, 3.6ms
6: 640x640 (no detections), 3.6ms
7: 640x640 (no detections), 3.6ms
8: 640x640 2 undamagedresidentialbuildings, 3.6ms
9: 640x640 (no detections), 3.6ms
10: 640x640 1 undamagedcommercialbuilding, 31 undamagedresidentialbuildings, 3.6ms
11: 640x640 (no detections), 3.6ms
12: 640x640 (no detections), 3.6ms
13: 640x640 1 undamagedcommercialbuilding, 3.6ms
14: 640x640 (no detections), 3.6ms
15: 640x640 5 undamagedresidentialbuildings, 3.6ms
16: 640x640 39 undamagedresidentialbuildings, 3.6ms
17: 640x640 7 undamagedresidentialbuildings, 3.6ms
18: 640x640 2 undamagedresidentialbuildings, 3.6ms
19: 640x640 48 undamagedresidentialbuildings, 3.6ms
20: 640x640 21 undamagedresidentialbuildings, 3.6ms
21: 640x640 41 undamagedresidenti

In [19]:
type(inference_results[0])

ultralytics.engine.results.Results

In [24]:
len(inference_results[4])

17

In [21]:
inference_results[1].names

{0: 'undamagedcommercialbuilding', 1: 'undamagedresidentialbuilding'}

In [ ]:
inference_results[1]

ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'undamagedcommercialbuilding', 1: 'undamagedresidentialbuilding'}
obb: None
orig_img: array([[[ 56,  91,  47],
        [ 55,  90,  46],
        [ 55,  90,  46],
        ...,
        [ 36,  54,  31],
        [ 37,  55,  32],
        [ 36,  54,  31]],

       [[ 56,  91,  47],
        [ 56,  91,  47],
        [ 57,  92,  48],
        ...,
        [ 28,  46,  23],
        [ 35,  53,  30],
        [ 39,  57,  34]],

       [[ 54,  89,  45],
        [ 55,  90,  46],
        [ 56,  91,  47],
        ...,
        [ 29,  47,  24],
        [ 36,  55,  30],
        [ 40,  59,  34]],

       ...,

       [[216, 220, 225],
        [215, 219, 224],
        [215, 219, 224],
        ...,
        [ 94,  96,  97],
        [ 92,  94,  95],
        [ 92,  94,  95]],

       [[216, 220, 225],
        [216, 220, 225],
        [216, 220, 225],
        ...,
       

In [27]:
inference_results[1].boxes.cls.cpu().numpy()

array([          1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,
                 1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,           1,
                 1,           1,           1,           1,           1,           1,           1], dtype=float32)

In [ ]:
inference_results[1].boxes.conf.cpu().numpy()

array([    0.86175,     0.78157,     0.57962,     0.49385,     0.42382,     0.29368,     0.23214,     0.21685,     0.20087], dtype=float32)

In [28]:
inference_results[1].path

'/content/images_inference/Pre_Event_tile_0_51.jpg'

In [36]:
classes_ids =  {class_name: class_id for class_id, class_name in inference_results[1].names.items()}
classes_ids

{'undamagedcommercialbuilding': 0, 'undamagedresidentialbuilding': 1}

In [33]:
inference_results[1].names

{0: 'undamagedcommercialbuilding', 1: 'undamagedresidentialbuilding'}

In [43]:
from collections import defaultdict

# images_containing_commercial_buildings : dict[str] = defaultdict(str)

images_containing_commercial_buildings: dict[str, dict[str, str]] = {}

temp = []

for i, inference_result in enumerate(inference_results):
  classes_ids =  {class_name: class_id for class_id, class_name in inference_result.names.items()}
  image_objects_detected = inference_result.boxes.cls.cpu().numpy()
  class_id = classes_ids['undamagedcommercialbuilding']
  if class_id in image_objects_detected:
    print(f'i: {i} - {inference_result.path} - {np.count_nonzero(image_objects_detected == class_id)}')
    temp.append(inference_result.path)


i: 0 - /content/images_inference/Pre_Event_tile_1_68.jpg - 1
i: 10 - /content/images_inference/Pre_Event_tile_0_25.jpg - 1
i: 13 - /content/images_inference/Pre_Event_tile_0_41.jpg - 1
i: 26 - /content/images_inference/Pre_Event_tile_1_45.jpg - 1
i: 28 - /content/images_inference/Pre_Event_tile_1_32.jpg - 1
i: 33 - /content/images_inference/Pre_Event_tile_1_87.jpg - 1
i: 34 - /content/images_inference/Pre_Event_tile_0_40.jpg - 2
i: 42 - /content/images_inference/Pre_Event_tile_0_91.jpg - 1
i: 82 - /content/images_inference/Pre_Event_tile_1_31.jpg - 1
i: 89 - /content/images_inference/Pre_Event_tile_1_88.jpg - 1
i: 99 - /content/images_inference/Pre_Event_tile_1_9.jpg - 1
i: 104 - /content/images_inference/Pre_Event_tile_1_39.jpg - 1
i: 107 - /content/images_inference/Pre_Event_tile_1_43.jpg - 1
i: 116 - /content/images_inference/Pre_Event_tile_0_114.jpg - 1
i: 117 - /content/images_inference/Pre_Event_tile_1_47.jpg - 1
i: 143 - /content/images_inference/Pre_Event_tile_1_66.jpg - 1
i: 1

In [45]:
import os
import shutil

In [ ]:
def copy_files(files_paths: list[str], source_dir: str, destination_dir: str):

  # Create a source directory
  os.makedirs(name=destination_dir, exist_ok=True)


In [ ]:
# Writing a scrypt that iterates over all the images in the list of results in model predict
# Iterate over all files, select the paths of results images which contain commercial buildings with high enough confidence > 0.8
# store all these paths in a list the use it to select these files in a folder on the local machine
# then upload them to roboflow for manual annotation and extension of the dataset

In [47]:
os.makedirs(name='image_paths_commercial_buildings', exist_ok=True)

In [48]:
dest_path = 'image_paths_commercial_buildings'

In [49]:
shutil.copy2('/content/images_inference/Pre_Event_tile_1_68.jpg', dest_path)

'image_paths_commercial_buildings/Pre_Event_tile_1_68.jpg'

In [37]:
len(inference_results)

232